In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from astropy.io import fits
from datetime import datetime as dt
from scipy.ndimage import interpolation as interp
from skimage.feature.register_translation import (register_translation, _upsampled_dft)

# Desenvolvimento de funções para redução de noite de observação - 31/03/2019

Criando função de organização de imagens em pastas, isso evitará da necessidade de uma função de seleção dentro de cada função específica, mantendo o mesmo tipo de imagem dentro de uma pasta.

Serão usados os dados coletados no OPD no dia 16 de outubro de 2018 para realização dos testes. Os algorítmos que serão desenvolvidos aqui NUNCA devem ser usados nos dados originais, pois ainda não foram testados. E de qualquer forma é sempre bom ter um backup dos dados originais.

In [2]:
os.chdir("18out16")

Criaremos uma pasta para os arquivos de suporte (que não são imagens fits)

Coletando nomes dos arquivos:

In [18]:
files = glob("*.fits")

In [ ]:
Coletando cabeçalhos:

In [20]:
headers = [fits.getheader(file) for  file in files]

Verificando os objetos observados com Keyword 'OBJECT'

In [53]:
objetos = set([objeto["OBJECT"] for objeto in headers])

In [54]:
objetos

{'HD 11667',
 'HR0509',
 'KL Aql',
 'M 11',
 'Mars',
 'NGC 1261',
 'NGC 362',
 'V0336 Aql',
 'bias',
 'flat',
 'lixo',
 'test'}

Criando estrutura de pastas, primeiramente para as imagens de calibração:

In [3]:
bias_folder = "bias"
flat_folder = "flat"

In [59]:
os.makedirs(bias_folder)
os.makedirs(flat_folder)

Vamos querer separar as outras imagens entre imagens de teste e de ciência logo vamos criar duas pastas para separa-las

In [4]:
science_folder = "science"
test_folder = "test"

In [62]:
os.makedirs(science_folder)
os.makedirs(test_folder)

Primeiro vamos separar bias e flat via a keyword 'OBJECT'

In [66]:
bias_images = [i["IMAGE"]+".fits" for i in headers if i["OBJECT"]=="bias"]
flat_images = [i["IMAGE"]+".fits" for i in headers if i["OBJECT"]=="flat"]

Criando função para movimentar listas de arquivos

In [5]:
def move_files(arquivos, destino):
    """"
    Move lista de arquivos para o destino especificado
    """
    quantidade = len(arquivos)
    print("Movendo", quantidade, "arquivos... \n")
    i = 1
    for arquivo in arquivos:
        print(arquivo, "====>", destino + "/" + arquivo, "(",i, "de",quantidade,")")
        os.rename(arquivo, destino + "/" + arquivo)
        i+=1
    print("\n")


Movimentando:

In [72]:
move_files(bias_images, bias_folder)
move_files(flat_images, flat_folder)

Agora vamos separar imagens de ciência e teste, começando pelas de ciencia, para isso precisamos atualizar a lista de cabeçalhos:

In [6]:
# Transformando em função para usar posteriormente
def get_headers_cwd():
    """"
    Retorna cabeçalhos dos arquivos na pasta atual
    """
    files = glob("*.fits")
    headers = [fits.getheader(file) for  file in files]
    return headers

headers = get_headers_cwd()

A lista de cabeçalhos deve ser atualizada sempre após a movimentação de arquivos.

In [96]:
science_images = [i["IMAGE"]+".fits" for i in headers if i["COMMENT"][0].split("'")[1].strip()=="science"]

Note que a expressão de seleção dessa vez ficou bastante grande, isso aconteceu devido á formatação dos cabeçalhos do OPD,
mais tarde devo criar uma função para simplificar essa expressão. 


Agora seguiremos para selecionar as imagens de teste (teste, aquisição, foco, etc) isso deve ser feito depois da movimentação das imagens de calibração pois englobará tudo o que não for ciência

In [97]:
test_images = [i["IMAGE"]+".fits" for i in headers if i["COMMENT"][0].split("'")[1].strip()!="science"]

Movimentando:

In [98]:
move_files(science_images, science_folder)
move_files(test_images, test_folder)

Agora vamos separar as imagens de ciencia e teste em objetos, definiremos duas funções, uma para criação de pastas por lista e outra para a separação de fato. Que serão juntas numa função maior

In [7]:
def mkdirs_list(list_of_directories):
    """"
    Create a list of directories in reference to the present directory
    """
    
    for i in list_of_directories:
        print("Criando diretório:", i)
        os.makedirs(i)
    print("\n")

In [124]:
def select_images(keyword, value):
    """"
    Seleciona arquivos da coleção de cabeçalhos baseados numa chave e valor
    """
    files = glob("*.fits")
    headers = get_headers_cwd()
    return [j for i,j in zip(headers,files) if i[keyword]==value]

def select_images_CommentFormat(keyword, value):
    """"
    Seleciona arquivos da coleção de cabeçalhos baseados numa chave e valor
    """
    headers = get_headers_cwd()
    return [i["IMAGE"]+".fits" for i in headers if i[keyword][0].split("'")[1].strip()==value]

In [9]:
def sep_by_object(folder = "./"):
    """"
    Separe files in a folders by object
    """
    current_folder = os.getcwd() #Recording the present folder to return later
    os.chdir(folder)
    
    headers = get_headers_cwd()
    objcts = set([obj["OBJECT"] for obj in headers]) #Creating set of objects
    
    print(objcts)
    # Creating directories
    mkdirs_list(objcts)
    
    for obj in objcts:
        move_files(select_images("OBJECT",obj), obj)
        headers = get_headers_cwd()
        
    os.chdir(current_folder)

Testando movimentação

In [136]:
sep_by_object(test_folder)

{'test', 'KL Aql', 'M 11', 'V0336 Aql', 'NGC 1261', 'lixo', 'Mars', 'HR0509', 'NGC 362', 'HD 11667'}


In [137]:
sep_by_object(science_folder)

{'V0336 Aql', 'KL Aql', 'Mars', 'M 11'}


Precisamos agora de uma função para separar imagens por filtro

In [65]:
def sep_by_filter(folder = "./"):
    """"
    Separe files in folders by filter
    """
    current_folder = os.getcwd() #Recording the present folder to return later
    os.chdir(folder)
    
    headers = get_headers_cwd()
    filters = set([obj["FILTER"] for obj in headers])#Creating set of objects
    
    print("Imagens nos filtros:\t", filters, "\n")
    # Creating directories
    mkdirs_list(filters)
    
    for f in filters:
        move_files(select_images("FILTER",f), f)
        headers = get_headers_cwd()
        
    os.chdir(current_folder)
    
def sep_object_by_filter(folder = "./"):
    """
    Dado caminho relativo para uma pasta cuja as sub-pastas são de objetos,
    separa as imagens dos objetos por filtro
    """
    
    current_folder = os.getcwd() #Recording the present folder to return later
    os.chdir(folder)
    
    objects_folder_list = os.listdir()
    
    for folder in objects_folder_list:
        print("Organizando imagens de ", folder, "\n")
        sep_by_filter(folder)
        print("\n")
    
    os.chdir(current_folder)

Testando função nos flats:

In [158]:
sep_by_filter(flat_folder)

{'B', 'V', 'I'}
Criando diretório: B
Criando diretório: V
Criando diretório: I
Copiando 21 arquivos
flat_038.fits ====> B/flat_038.fits ( 1 de 21 )
flat_052.fits ====> B/flat_052.fits ( 2 de 21 )
flat_036.fits ====> B/flat_036.fits ( 3 de 21 )
flat_034.fits ====> B/flat_034.fits ( 4 de 21 )
flat_049.fits ====> B/flat_049.fits ( 5 de 21 )
flat_047.fits ====> B/flat_047.fits ( 6 de 21 )
flat_046.fits ====> B/flat_046.fits ( 7 de 21 )
flat_035.fits ====> B/flat_035.fits ( 8 de 21 )
flat_037.fits ====> B/flat_037.fits ( 9 de 21 )
flat_032.fits ====> B/flat_032.fits ( 10 de 21 )
flat_045.fits ====> B/flat_045.fits ( 11 de 21 )
flat_039.fits ====> B/flat_039.fits ( 12 de 21 )
flat_042.fits ====> B/flat_042.fits ( 13 de 21 )
flat_040.fits ====> B/flat_040.fits ( 14 de 21 )
flat_050.fits ====> B/flat_050.fits ( 15 de 21 )
flat_041.fits ====> B/flat_041.fits ( 16 de 21 )
flat_048.fits ====> B/flat_048.fits ( 17 de 21 )
flat_043.fits ====> B/flat_043.fits ( 18 de 21 )
flat_044.fits ====> B/flat_

Agora podemos gerar os master bias e flats. Antes iremos definir uma função para retornar feedback quanto ao master em tela.

In [68]:
def imstat(fits_file):
    """"
    Printa em tela e retorna estatísticas da imagem em um dicionário
    """
    image = fits.getdata(fits_file)
    print("Imagem: %s" % (fits_file))
    print("Median: " + str(np.median(image)) + " counts")
    print("Mean: " + str(np.mean(image)) + " counts")
    print("Std: " + str(np.std(image)) + " counts")   
    print("Max value: " + str(np.max(image)) + " counts")
    print("Min value: " + str(np.min(image)) + " counts")
    print("\n")
    
    return {"median":np.median(image), 
            "mean":np.mean(image), 
            "std":np.std(image),
            "max":np.max(image),
            "min":np.min(image)
           }


In [28]:
def make_MasterBias(bias_folder, out_folder):
    """"
    Dado o caminho relativo a pasta com os bias gera master bias e salva no caminho relativo especificado
    """
    current_folder = os.getcwd() #Recording the present folder to return later
    os.chdir(bias_folder)
    
    out = out_folder+"master_bias.fits"
    files = os.listdir()
    ref_header = fits.getheader(files[0])
    NCOMBINE = len(files)
    
    print("Carregando e empilhando %i imagens ... \n" % (NCOMBINE))
    bias_cube = np.stack([fits.getdata(image) for image in files],axis=0)
    
    print("Extraindo medianas ... \n")
    master_bias = np.median(bias_cube, axis=0)
    
    print("Escrevendo FITS ... \n")
    fits.writeto(out, master_bias, ref_header)
    
    print("Gerado master bias: %s \n Estatistica de contagens: \n" % out)
    imstat(out)
    
    #Escrevendo meta-dados no cabeçalho
    with fits.open(out, mode="update") as filehandle:
        filehandle[0].header["NCOMBINE"] = NCOMBINE
        
    os.chdir(current_folder)
    
    return out

Perceba que podemos transformar essa função em uma geradora de dark facilmente. Testando função:

In [196]:
make_MasterBias(bias_folder)

Median: 853.0 counts
Mean: 853.38210105896 counts
STD: 0.7321106926931815 counts
Max value: 857.0 counts
Min value: 850.0 counts


In [101]:
def make_MasterFlat(flat_folder, out_folder, master_bias):
    """"
    Dada localização relativa da pasta de flats e o master bias, gera arquivos de flat master
    """
    current_folder = os.getcwd() #Recording the present folder to return later
    os.chdir(flat_folder)
    
    files = os.listdir()
    ref_header = fits.getheader(files[0])
    band = ref_header["FILTER"]
    
    master_bias = fits.getdata(master_bias)
    NCOMBINE = len(files)
    
    flat_cube = np.stack([fits.getdata(image) for image in files],axis=0)
    
    master_flat = np.median(flat_cube, axis=0) - master_bias
    
    mean = np.mean(master_flat)
    
    norm_master_flat = master_flat / mean
    
    file_name = out_folder + "master_flat_%s.fits" % (band)
    
    fits.writeto(file_name, norm_master_flat, ref_header)
    
    imstat(file_name)
    
    #Escrevendo meta-dados no cabeçalho
    with fits.open(file_name, mode="update") as filehandle:
        filehandle[0].header["NCOMBINE"] = NCOMBINE
        
    os.chdir(current_folder)
    
    return file_name
    
def make_MasterFlat_all(flats_folder, out_folder, master_bias):
    """
    Dado caminho relativo da pasta de flats aplica função de master flat para cada filtro 
    (dado que foram separados em pastas)
    """
    
    current_folder = os.getcwd() #Recording the present folder to return later
    
    os.chdir(flats_folder)
    sub_folders = os.listdir()
    
    mflat_dict = {}
    
    for folder in sub_folders:
        mflat_dict[folder] = make_MasterFlat(folder, out_folder, master_bias)
        print("\n")
        
    os.chdir(current_folder)
    
    return mflat_dict
    

Note que a função não leva um dark em consideração pois no OPD usamos um CCD refrigerado. Note também que estamos usando a mediana, pois temos o costume de coletar imagens de calibração com número impar.

Testando função:

In [44]:
make_MasterFlat_all('flat/', "../../bias/master_bias.fits")

Median: 1.0028433668611008 counts
Mean: 1.0000000000000009 counts
Std: 0.03182351866459908 counts
Max value: 1.1595145032418743 counts
Min value: 0.006972760303673663 counts
Median: 1.0024265808197768 counts
Mean: 0.9999999999999996 counts
Std: 0.032357645299030645 counts
Max value: 1.19121765292956 counts
Min value: 0.06431206182315967 counts
Median: 1.0029264849504804 counts
Mean: 0.9999999999999999 counts
Std: 0.03226009755435671 counts
Max value: 1.1859490754081279 counts
Min value: 0.005330108204081474 counts


Ao final da separação dos arquivos devemos fazer um back-up dos dados de ciência pois agora começaremos a modificar os dados.

In [18]:
os.listdir("science/M 11")

['V', 'I', 'B']

In [29]:
os.getcwd()
sep_object_by_filter()

Organizando imagens de  M 11 

Imagens nos filtros:	 {'V', 'B', 'I'} 

Criando diretório: V
Criando diretório: B
Criando diretório: I


Movendo 20 arquivos... 

OI2018B-011_2018-10-16_190.fits ====> V/OI2018B-011_2018-10-16_190.fits ( 1 de 20 )
OI2018B-011_2018-10-16_196.fits ====> V/OI2018B-011_2018-10-16_196.fits ( 2 de 20 )
OI2018B-011_2018-10-16_169.fits ====> V/OI2018B-011_2018-10-16_169.fits ( 3 de 20 )
OI2018B-011_2018-10-16_168.fits ====> V/OI2018B-011_2018-10-16_168.fits ( 4 de 20 )
OI2018B-011_2018-10-16_194.fits ====> V/OI2018B-011_2018-10-16_194.fits ( 5 de 20 )
OI2018B-011_2018-10-16_191.fits ====> V/OI2018B-011_2018-10-16_191.fits ( 6 de 20 )
OI2018B-011_2018-10-16_167.fits ====> V/OI2018B-011_2018-10-16_167.fits ( 7 de 20 )
OI2018B-011_2018-10-16_192.fits ====> V/OI2018B-011_2018-10-16_192.fits ( 8 de 20 )
OI2018B-011_2018-10-16_198.fits ====> V/OI2018B-011_2018-10-16_198.fits ( 9 de 20 )
OI2018B-011_2018-10-16_193.fits ====> V/OI2018B-011_2018-10-16_193.fits ( 10 de 20 

In [25]:
def ccdproc(image_file, out_path, master_bias, master_flat):
    """"
    Dado caminho para imagem de determinado objeto em
    determinado filtro aplica imagens de calibração
    """
    mbias = fits.getdata(master_bias)
    mflat = fits.getdata(master_flat)
    
    header = fits.getheader(image_file)
    image = fits.getdata(image_file)
    
    print("Processando imagem: %s ..." % (image_file) )
    
    proc_image = (image - mbias) / mflat
    
    out = out_path + "r_%s" % (image_file)
    
    #Updating header
    
    now = dt.now().strftime("%B %d, %Y")
    
    header["CCDProc"] = "Done: %s" % (now)
    
    fits.writeto(out, proc_image, header)

In [18]:
ccdproc("OI2018B-011_2018-10-16_140.fits", "../../bias/master_bias.fits", "../../flat/V/master_flat_V.fits")

Processando imagem: OI2018B-011_2018-10-16_140.fits


In [24]:
def ccdproc_all(folder_path, out_path, master_bias, master_flat):
    """"
    Dado caminho para imagem de determinado objeto em
    determinado filtro aplica imagens de calibração indicadas
    para bias e no dicionário de flats.
    """
     
    current_folder = os.getcwd() 
    os.chdir(folder_path)     
    
    images = os.listdir()
    images.sort()
    
    ref_header = fits.getheader(images[0])
    band = ref_header["FILTER"]
    
    
    N = len(images)
    
    print("Processando %i imagens no filtro %s" % (N, band))
    
    for image in images:
        ccdproc(image, out_path, master_bias, master_flat)
        
    os.chdir(current_folder)

In [98]:
def ccdproc_all_filters(folder_path, out_path, master_bias, master_flat_dict):
    
    current_folder = os.getcwd() 
    os.chdir(folder_path)  
    
    filters = os.listdir()
    
    for band in filters:
        master_flat = master_flat_dict[band]
        ccdproc_all(band, out_path, master_bias, master_flat)
        print("\n")
        
    os.chdir(current_folder)

In [85]:
# Making test dict
root = "/home/joaopedro/Documents/reduction_pipeline_devel/18out16/"
flat_dict = {"B": str(root+"flat/B/master_flat_B.fits"), 
             "V": str(root+"flat/V/master_flat_V.fits"),
             "I": str(root+"flat/I/master_flat_I.fits")}

Testando:

In [58]:
ccdproc_all("M 11/B", root+"bias/master_bias.fits", root+"flat/B/master_flat_B.fits")

Processando 30 imagens no filtro B
Processando imagem: OI2018B-011_2018-10-16_144.fits ...
Processando imagem: OI2018B-011_2018-10-16_147.fits ...
Processando imagem: OI2018B-011_2018-10-16_181.fits ...
Processando imagem: OI2018B-011_2018-10-16_159.fits ...
Processando imagem: OI2018B-011_2018-10-16_186.fits ...
Processando imagem: OI2018B-011_2018-10-16_189.fits ...
Processando imagem: OI2018B-011_2018-10-16_154.fits ...
Processando imagem: OI2018B-011_2018-10-16_152.fits ...
Processando imagem: OI2018B-011_2018-10-16_151.fits ...
Processando imagem: OI2018B-011_2018-10-16_158.fits ...
Processando imagem: OI2018B-011_2018-10-16_150.fits ...
Processando imagem: OI2018B-011_2018-10-16_188.fits ...
Processando imagem: OI2018B-011_2018-10-16_157.fits ...
Processando imagem: OI2018B-011_2018-10-16_140.fits ...
Processando imagem: OI2018B-011_2018-10-16_183.fits ...
Processando imagem: OI2018B-011_2018-10-16_153.fits ...
Processando imagem: OI2018B-011_2018-10-16_146.fits ...
Processando i

Funciona !! Seguimos testando a próxima função

In [132]:
os.chdir("../")

In [135]:
sep_object_by_filter("reduced")

Organizando imagens de  HD 11667 

Imagens nos filtros:	 {'B', 'I', 'V'} 

Criando diretório: B
Criando diretório: I
Criando diretório: V


Movendo 6 arquivos... 

r_OI2018B-011_2018-10-30_0163.fits ====> B/r_OI2018B-011_2018-10-30_0163.fits ( 1 de 6 )
r_OI2018B-011_2018-10-30_0194.fits ====> B/r_OI2018B-011_2018-10-30_0194.fits ( 2 de 6 )
r_OI2018B-011_2018-10-30_0196.fits ====> B/r_OI2018B-011_2018-10-30_0196.fits ( 3 de 6 )
r_OI2018B-011_2018-10-30_0164.fits ====> B/r_OI2018B-011_2018-10-30_0164.fits ( 4 de 6 )
r_OI2018B-011_2018-10-30_0165.fits ====> B/r_OI2018B-011_2018-10-30_0165.fits ( 5 de 6 )
r_OI2018B-011_2018-10-30_0195.fits ====> B/r_OI2018B-011_2018-10-30_0195.fits ( 6 de 6 )


Movendo 3 arquivos... 

r_OI2018B-011_2018-10-30_0171.fits ====> I/r_OI2018B-011_2018-10-30_0171.fits ( 1 de 3 )
r_OI2018B-011_2018-10-30_0169.fits ====> I/r_OI2018B-011_2018-10-30_0169.fits ( 2 de 3 )
r_OI2018B-011_2018-10-30_0170.fits ====> I/r_OI2018B-011_2018-10-30_0170.fits ( 3 de 3 )


Movendo

Esse também funciona. Com isso posso desenvolver completamente o algorítimo de processamento inicial

Para agilizar o processo essas funções agora devem ser organizadas num método global

In [136]:
def initial_reduction(observation_folder = "./"):
    """"
    Organiza arquivos de observações em pasta, levando em consideração informações do header no padrão do LNA.
    Dado caminho relativo da pasta.
    """
    
    os.chdir(observation_folder)
    
    # Definindo nomes das pastas para estrutura de arquivos (referente a pasta atual)
    root = os.getcwd()
    bias_folder = root + "/calibration/bias/"
    flat_folder = root + "/calibration/flat/"
    master_folder = root +"/calibration/master/"
    others_folder = root +"/others/"
    raw_science_folder = root + "/science/raw/"
    reduced_science_folder = root + "/science/reduced/"
    
    folders_list = [bias_folder, flat_folder, master_folder, others_folder, raw_science_folder, reduced_science_folder]
    
    # Depois de definida a estrutura de pastas podemos cria-las
    
    print("Criando estrutura de pastas ... \n")
    
    mkdirs_list(folders_list)
    
    # Agora vamos olhar os arquivos dentro da pasta e organizar as imagens de calibração (bias e flat)
    
    headers = get_headers_cwd()
    N = len(headers)
    
    bias_images = select_images("OBJECT", "bias")
    flat_images = select_images("OBJECT", "flat")
    
    print("Organizando arquivos de calibração ... \n")
    
    move_files(bias_images, bias_folder)
    move_files(flat_images, flat_folder)
    
    print("Separando flats por filtro ... \n")
    
    sep_by_filter(flat_folder)
    
    # Seguimos agora organizando o resto das imagens em duas categorias (science e others)
    # Atualizando a seleção de imagens
    
    headers = get_headers_cwd()
    
    science_images = [i["IMAGE"]+".fits" for i in headers if i["COMMENT"][0].split("'")[1].strip()=="science"]
    other_images = [i["IMAGE"]+".fits" for i in headers if i["COMMENT"][0].split("'")[1].strip()!="science"]
    
    # A função select_images() não funciona nesse caso devido a formatação do comentário no cabeçalho e a expressão !=.
    # No futuro posso deixa-la mais robusta
    
    # Continuando com a organização ...
    
    print("Organizando imagens de ciência e outras ... \n")
    
    move_files(science_images, raw_science_folder)
    move_files(other_images, others_folder)
    
    print("Separando imagens por objeto ... \n")
    
    sep_by_object(raw_science_folder)
    sep_by_object(others_folder)
    
    print("Separando imagens de ciência dos objetos em filtros ... \n")
    
    sep_object_by_filter(raw_science_folder)
    
    # Agora estamos com todos os arquivos organizados, vamos seguir gerando os arquivos de calibração.
    
    print("Gerando arquivos de calibração ... \n")
    
    print("Gerando Master Bias ... \n")
    
    mbias = make_MasterBias(bias_folder, master_folder)
    
    # A função make_MasterBias() imprime em tela uma boa quantidade de feedback a cada passo para termos noção
    # de quanto tempo cada passo da função está demorando para rodar.
    
    print("Gerando Master Flats ... \n")
    
    mflat_dict = make_MasterFlat_all(flat_folder, master_folder, mbias) 
    
    # Agora que temos as imagens de calibração vamos seguir calibrando as imagens de ciência
    
    # Vamos para a pasta de imagens de ciência para ver o que temos
    
    print("Aplicando arquivos de calibração nas imagens de ciência ... \n ")
    
    os.chdir(raw_science_folder)
    objcts = os.listdir()
    
    for objct in objcts:
        print("Aplicando calibrações para %s ... \n" % objct)
        ccdproc_all_filters(objct, reduced_science_folder, mbias, mflat_dict)
        
    os.chdir(root)
    
    # Imagens agora processadas na pasta de imagens reduzidas, por simplicidade elas foram salvas
    # todas juntas, agora vamos separa-las novamente.
    
    print("Fazendo organização dos arquivos reduzidos ..")
    
    sep_by_object(reduced_science_folder)
    sep_object_by_filter(reduced_science_folder)
    
    # Finalmente as imagens estão organizadas e corrigidas para erros instrumentais.
    
    print("Redução conluída. Foram processados %i arquivos fits." % (N))
    


Vamos fazer um teste geral!!

In [111]:
os.chdir("../")
os.getcwd()

'/home/joaopedro/Documents/reduction_pipeline_devel'

In [97]:
ccdproc_all_filters("HD 11667/", "../../test/", "/home/joaopedro/Documents/reduction_pipeline_devel/testing/calibration/master/master_bias.fits", flat_dict)

Processando 6 imagens no filtro V
Processando imagem: OI2018B-011_2018-10-30_0166.fits ...
Processando imagem: OI2018B-011_2018-10-30_0167.fits ...
Processando imagem: OI2018B-011_2018-10-30_0168.fits ...
Processando imagem: OI2018B-011_2018-10-30_0197.fits ...
Processando imagem: OI2018B-011_2018-10-30_0198.fits ...
Processando imagem: OI2018B-011_2018-10-30_0199.fits ...
Processando 3 imagens no filtro I
Processando imagem: OI2018B-011_2018-10-30_0169.fits ...
Processando imagem: OI2018B-011_2018-10-30_0170.fits ...
Processando imagem: OI2018B-011_2018-10-30_0171.fits ...
Processando 6 imagens no filtro B
Processando imagem: OI2018B-011_2018-10-30_0163.fits ...
Processando imagem: OI2018B-011_2018-10-30_0164.fits ...
Processando imagem: OI2018B-011_2018-10-30_0165.fits ...
Processando imagem: OI2018B-011_2018-10-30_0194.fits ...
Processando imagem: OI2018B-011_2018-10-30_0195.fits ...
Processando imagem: OI2018B-011_2018-10-30_0196.fits ...


In [140]:
initial_reduction("testing")

Criando estrutura de pastas ... 

Criando diretório: /home/joaopedro/Documents/reduction_pipeline_devel/testing/calibration/bias/
Criando diretório: /home/joaopedro/Documents/reduction_pipeline_devel/testing/calibration/flat/
Criando diretório: /home/joaopedro/Documents/reduction_pipeline_devel/testing/calibration/master/
Criando diretório: /home/joaopedro/Documents/reduction_pipeline_devel/testing/others/
Criando diretório: /home/joaopedro/Documents/reduction_pipeline_devel/testing/science/raw/
Criando diretório: /home/joaopedro/Documents/reduction_pipeline_devel/testing/science/reduced/


Organizando arquivos de calibração ... 

Movendo 21 arquivos... 

bias_0012.fits ====> /home/joaopedro/Documents/reduction_pipeline_devel/testing/calibration/bias//bias_0012.fits ( 1 de 21 )
bias_0008.fits ====> /home/joaopedro/Documents/reduction_pipeline_devel/testing/calibration/bias//bias_0008.fits ( 2 de 21 )
bias_0011.fits ====> /home/joaopedro/Documents/reduction_pipeline_devel/testing/calibr

In [139]:
os.chdir("../")
os.getcwd()

'/home/joaopedro/Documents/reduction_pipeline_devel'